In [2]:
from __future__ import print_function
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as plt
import subprocess

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
def print_top_words(tf_idf, feature_names, n_top_words):
    for topic_idx, topic in enumerate(tf_idf):
        #print(topic.shape)
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [6]:
'''
w = len('Abstract')
pids = filtered.loc[filtered.ABSTRACT==' '].index
for p in pids:
    t = filtered.loc[p,'TITLE']
    print (t)
    cmd = "python /home/yu/gits/scholar/scholar.py -c 1 -t --phrase '"+t+"'"+" --csv"
    print (cmd)
    result = subprocess.check_output(cmd, shell=True)
    print (result)
    filtered.loc[p,'ABSTRACT'] = result.split('|')[-1][w:]
    break
'''

'\nw = len(\'Abstract\')\npids = filtered.loc[filtered.ABSTRACT==\' \'].index\nfor p in pids:\n    t = filtered.loc[p,\'TITLE\']\n    print (t)\n    cmd = "python /home/yu/gits/scholar/scholar.py -c 1 -t --phrase \'"+t+"\'"+" --csv"\n    print (cmd)\n    result = subprocess.check_output(cmd, shell=True)\n    print (result)\n    filtered.loc[p,\'ABSTRACT\'] = result.split(\'|\')[-1][w:]\n    break\n'

In [81]:
publications = pd.read_csv('filtered_outputs.csv')
people = pd.read_csv('people_orgs.csv')

In [87]:
publications['topic'] = -1

In [64]:
topic_years = pd.DataFrame(columns=['y'+str(y) for y in range(2008,2014)])

,y2008,y2009,y2010,y2011,y2012


In [96]:
tf_vectorizer = CountVectorizer(max_df=0.7, min_df=2,
                                max_features=10000,
                                stop_words='english')

In [97]:
T = 50
lda = LatentDirichletAllocation(n_topics=T, max_iter=30,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0,
                                n_jobs=1)

In [99]:
for year in range(2008,2014):
    print(year)
    filtered = publications.loc[publications.PUBLICATION_YEAR==year]
    docs = map(lambda i: filtered.TITLE[i]+filtered.ABSTRACT[i], filtered.index)
    
    doc_word_mtrx = tf_vectorizer.fit_transform(docs)
    for w in tf_vectorizer.vocabulary_.keys():
        if len(w) < 3 or any(x.isdigit() for x in w):
            del tf_vectorizer.vocabulary_[w]
    doc_word_mtrx = tf_vectorizer.transform(docs)
    
    df = reduce(lambda x,y:x+y, doc_word_mtrx)
    df = df.toarray()
    df[df==0]=np.max(df)
    
    doc_topic=lda.fit_transform(doc_word_mtrx)
    dp = np.argsort(doc_topic,1)[:,-1]
    tp_doc_counts=np.zeros(T)
    for t in range(T):
        tp_doc_counts[t] = (dp==t).sum()
        
    trank = tp_doc_counts.argsort()
    n_top_words = 50
    tf_feature_names = tf_vectorizer.get_feature_names()
    for t in range(T):
        topic_years.loc[t,'y'+str(year)] = ",".join(tf_feature_names[k] for k in lda.components_[trank[-t-1]].argsort()[:-n_top_words - 1:-1])
        publications.loc[filtered.index[dp==trank[-t-1]],'topic'] = t

2008
2009
2010
2011
2012
2013


In [108]:
topic_years.to_csv('topic_years.csv')

In [111]:
people.PERSON_ID.unique().shape

(3263,)

In [115]:
people_topic = pd.merge(publications, people, on='PUBLICATION_ID')

In [133]:
topic_size = pd.DataFrame()
for year in range(2008,2014):
    #topic_orgs[year] = {}
    ycol = 'y'+str(year)
    for t in range(T):
        topic_size.loc[t,ycol+'_num_people'] = people_topic.loc[(people_topic.PUBLICATION_YEAR==year)&(people_topic.topic==t)].PERSON_ID.unique().shape[0]
        topic_size.loc[t,ycol+'_num_orgs'] = people_topic.loc[(people_topic.PUBLICATION_YEAR==year)&(people_topic.topic==t)].ORGANISATION_CODE.unique().shape[0]
        topic_size.loc[t,ycol+'_num_docs'] = people_topic.loc[(people_topic.PUBLICATION_YEAR==year)&(people_topic.topic==t)].PUBLICATION_ID.unique().shape[0]
        #topic_orgs[year][t]={'num_people':num_people,'num_orgs':num_orgs,'num_docs':num_docs}
        

In [134]:
topic_size.to_csv('topic_size.csv',index=False,float_format='%d')

In [149]:
topic_orgs = pd.DataFrame()
orgs = people_topic.ORGANISATION_CODE.unique()
N = len(orgs)
count = np.zeros(N)
for year in range(2008,2014):    
    ycol = 'y'+str(year)
    f = people_topic.loc[(people_topic.PUBLICATION_YEAR==year)]
    for t in range(T):
        ff = f.loc[(f.topic==t)]
        count*=0
        for n,o in enumerate(orgs):
            count[n] = ff.loc[(ff.ORGANISATION_CODE==o)].PUBLICATION_ID.unique().shape[0]
        topic_orgs.loc[t,ycol] = ','.join(org for org in orgs[count.argsort()[-5:]])+' ('+','.join(str(int(c)) for c in np.sort(count)[-5:])+')'
        

In [150]:
topic_orgs.head()

,y2008,y2009,y2010,y2011,y2012,y2013
0,"SPOL,QUEN,MVEN,SOCS,SSCM (106,138,144,154,406)","GELY,CHEM,PHYS,MVEN,QUEN (98,152,171,318,387)","SPAI,LAWD,EDUC,SOCS,SSCM (98,103,112,124,250)","GELY,CHEM,PHYS,MVEN,QUEN (126,127,153,274,305)","VESC,EDUC,SPOL,SOCS,SSCM (88,90,110,158,395)","GEOG,PHYS,MATH,MVEN,QUEN (76,88,120,292,427)"
1,"MVEN,GELY,PHYS,QUEN,CHEM (60,111,116,146,176)","BISC,PSYC,VESC,SOCS,SSCM (44,65,84,190,294)","GELY,CHEM,PHYS,MVEN,QUEN (49,100,126,217,329)","SPAI,GEOG,SSCM,HUMS,EDUC (72,81,85,92,107)","GELY,MVEN,CHEM,PHYS,QUEN (91,119,133,166,347)","SART,LAWD,SPAI,GEOG,HUMS (67,70,93,94,109)"
2,"SSCM,PANM,BIOC,PHPH,SOCS (27,36,54,82,143)","SSCM,LAWD,SPAI,SPOL,EDUC (38,45,48,58,114)","MATH,GEOG,SOCS,MVEN,SSCM (75,78,79,96,140)","VESC,PSYC,SPOL,SOCS,SSCM (59,61,85,179,408)","BISC,PHYS,MATH,QUEN,MVEN (32,34,68,78,191)","QUEN,GEOG,GELY,PHYS,CHEM (47,115,138,147,177)"
3,"HUMS,PHYS,MATH,MVEN,QUEN (31,38,49,64,86)","CHEM,PANM,PHPH,BIOC,SOCS (25,44,80,80,105)","BISC,PANM,BIOC,PHPH,SOCS (30,69,71,91,147)","BIOC,SSCM,PHPH,VESC,SOCS (54,69,82,90,225)","VESC,PANM,PHPH,BIOC,SOCS (42,62,73,90,196)","VESC,EDUC,SPOL,SOCS,SSCM (43,44,55,157,285)"
4,"SPAI,LAWD,MODL,EDUC,HUMS (27,28,30,60,65)","BISC,ORDS,VESC,SOCS,SSCM (22,22,53,54,111)","PHPH,SSCM,SOCS,VESC,CHEM (8,11,31,46,63)","BIOC,VICH,DPPP,PHYS,CHEM (7,8,11,53,110)","QUEN,BISC,GEOG,CHEM,GELY (20,21,86,92,123)","SPOL,PSYC,VESC,SOCS,SSCM (34,39,60,148,341)"


In [152]:
topic_orgs.to_csv('topic_orgs.csv',index=False)